# Data Mining Versuch Recommender Systeme

* Autor: Prof. Dr. Johannes Maucher
* Datum: 30.09.2015

## Abgabe:

- **Abzugeben ist das Jupyter Notebook mit dem verlangten Implementierungen und den entsprechenden Ausgaben.**
- **Das Notebook ist als .ipynb und als .html abzugeben.**
- **Klausurelevante Fragen sind Dokument "Fragenkatalog Datamining" zu finden.**
- Antworten auf Fragen im Notebook, Diskussionen und Beschreibung der Ergebnisse sind optional (aber empfohlen) und werden nicht bewertet.

* [Übersicht Data Mining Praktikum](https://maucher.pages.mi.hdm-stuttgart.de/ai/page/dm/)


# Einführung

## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __Ähnlichkeit:__ Verfahren zur Bestimmung der Ähnlichkeit zwischen Personen (Kunden) und Elementen (Produkten)
* __Empfehlungssysteme__ Collaborative Filtering 
* __Collaborative Filtering:__ Nutzerbezogener Ansatz und elementbasierter Ansatz

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung

### Recommender Systeme

Recommender Systeme werden im E-Commerce eingesetzt um Werbung in Form von kundenspezifischen Empfehlungen zu verteilen. Weitläufig bekannt sind die Amazon-Empfehlungen, die entweder per e-mail geschickt oder nach dem Log-In in der Web-Page angezeigt werden. Diese Empfehlungen werden in Abhängigkeit von den bisher vom jeweiligen Kunden gekauften bzw. bewerteten Produkten erstellt. In diesem Versuch werden die derzeit wohl am weitest verbreiteteten Verfahren für die Erzeugung kundenspezifischer Empfehlungen vorgestellt, darunter das elementweise Collaborative Filtering, welches z.B. auch von Amazon eingesetzt wird.     

Direkt-Marketing Methoden wie die kundenspezifische Erzeugung und Bereitstellung von Werbung erfordern detaillierte Kunden- und Warenkorbanalysen. Kunden mit ähnlichem Kaufverhalten werden in Kundengruppen zusammengefasst. Die Warenkorbanalyse untersucht u.a. welche Waren bevorzugt im Verbund von der gleichen Person gekauft werden. Damit kann ein Händler Werbung in Form von Empfehlungen individuell und gezielt an seine Kunden richten, abhängig davon welcher Kundengruppe er angehört und welche Produkte bevorzugt von dieser Kundengruppe nachgefragt werden. 

Im ersten Teil der Übung werden fiktive Daten in einer überschaubaren Menge verwendet. Es handelt sich hier um Filmbewertungen. Anhand dieses Beispiels sollen die notwendigen Methoden und Abläufe implementiert und getestet werden. Diese werden im zweiten Teil der Übung auf echte Daten angewandt. Hierzu werden über eine Python-API Daten vom Internet-Meta-Radio _last.fm_ integriert. Auf der Basis dieser Daten sollen dann Musikempfehlungen für last.fm User berechnet werden. 

Recommender Systeme lassen sich mit

* Clustering Verfahren
* Suchalgorithmen
* Collaborativen Filtering 
 
realisieren. Am häufigsten wird hierbei das Collaborative Filtering eingesetzt. Für das Collaborative Filtering wird jeder der $M$ User durch einen $N$-dimensionalen Vektor beschrieben, wobei $N$ die Anzahl der Produkte im Angebot des Händlers ist. Jedes Element im Vektor gehört zu einem speziellen Produkt. Das Element hat den Wert 1, wenn der User dieses Produkt bereits gekauft hat, sonst 0 (andere Wertbelegungen sind möglich, z.B. wenn Produktbewertungen vorliegen). Alle $M$ Zeilenvektoren können zur _User/Item_ Matrix zusammengefasst werden (siehe Abbildung).


<img src="https://maucher.home.hdm-stuttgart.de/Pics/UserItemMatrix.png" style="width:1000px" align="center">

Das traditionelle **userbasierte Collaborative Filtering (UCF)**, benutzt die Ähnlichkeit zwischen Benutzern: Um für User $U_i$ eine Empfehlung zu erzeugen wird zunächst der diesem User ähnlichste Kunde (oder eine Menge vom ähnlichsten Kunden) ermittelt. Dann werden $U_i$ die Produkte (Items) empfohlen, welche der ähnlichste Kunde gekauft hat, $U_i$ selbst jedoch noch nicht. 

Dieser Ansatz skaliert schlecht im Fall sehr großer *User/Item*-Matrizen. Ausserdem ist er für User, welche erst wenige Produkte gekauft haben unzuverlässig. Besser eignet sich in diesen Fällen das **itembasierte Collaborative Filtering (ICF)**. Es wird u.a. von Amazon.com eingesetzt. Diese Variante benutzt die Ähnlichkeit zwischen Produkten (Items). Dabei sind Produkte umso ähnlicher je mehr Kunden diese Produkte gemeinsam gekauft haben. Für die Produkte welche ein Referenzuser $U_i$ bereits gekauft hat, werden die ähnlichsten Produkte ermittelt. Diese ähnlichsten Produkte werden $U_i$ empfohlen, wenn er sie nicht schon selbst gekauft hat.

Im folgenden Abschnitt werden einige gebräuchliche Metriken für die Berechnung der Ähnlichkeit zwischen Benutzern oder Artikeln vorgestellt. Für Collaboratives Filtering wird sehr häufig das Cosinus - Ähnlichkeitsmaß eingesetzt.


### Gebräuchliche Ähnlichkeitsmaße

Die __euklidische Distanz__ $d_E(\underline{a},\underline{b})$ zwischen zwei n-dimensionalen Vektoren $\underline{a}=(a_1,\ldots,a_n)$ und $\underline{b}=(b_1,\ldots,b_n)$ berechnet sich zu
	$$
	d_E(\underline{a},\underline{b})=\sqrt{\sum_{i=1}^n (a_i-b_i)^2}
	$$
Zwei Vektoren können als umso ähnlicher erachtet werden, je kleiner deren euklidische Distanz ist. 
Ein auf der euklidischen Metrik basierendes Ähnlichkeitsmaß zwischen zwei Vektoren $\underline{a}$ und $\underline{b}$ kann durch 
$$
s_E(\underline{a},\underline{b})=\frac{1}{1+d_E(\underline{a},\underline{b})}
$$
angegeben werden.


__Pearson Korrelation__
Die Ähnlichkeit zwischen zwei Vektoren kann auch durch den Pearson-Korrelationskoeffizient $\rho_{\underline{a},\underline{b}}$ ausgedrückt werden. Er berechnet sich zu
$$
\rho_{\underline{a},\underline{b}}= \frac{1}{N}\cdot \sum\limits_{i=1}^{N}\frac{(a_i-\overline{a})}{\sigma_a} \frac{(b_i-\overline{b})}{\sigma_b}
$$
Dabei bezeichnet $N$ die Länge der Vektoren, $\overline{a}$ den Mittelwert und $\sigma_a$ die Standardabweichung des Vektors $\underline{a}$. 

Der Pearson-Korrelationskoeffizient misst die lineare Abhängigkeit zwischen zwei Vektoren. Der maximale Wert von $+1$ wird erreicht, wenn die durch die beiden Vektoren definierten N Punkte im 2-dimensionalen Raum auf einer ansteigenden Geraden liegen. Der Minimalwert von $-1$ wird erreicht, wenn die Punkte auf einer abfallenden Geraden liegen. Der Betrag des Koeffizienten ist umso kleiner, je stärker die Punkte von einer fiktiven Geraden (kann durch lineare Regression berechnet werden) abweichen. Der Koeffizient ist $0$ wenn keine lineare Abhängigkeit zwischen den Vektoren besteht.


__Cosinus Ähnlichkeitsmaß__
Die Ähnlichkeit zwischen zwei Vektoren kann auch durch den Cosinus $\cos(\underline{a},\underline{b})$ ausgedrückt werden. Er berechnet sich zu
$$
\cos(\underline{a},\underline{b})= \frac{\underline{a} \cdot \underline{b}}{\left\|\underline{a}\right\|\cdot \left\|\underline{b}\right\|}
$$
wobei im Zähler das Skalarprodukt der beiden Vektoren steht und mit $\left\|\underline{x}\right\|$ der Betrag des Vektors $\underline{x}$ bezeichnet wird.

Falls die Vektoren $\underline{a}$ und $\underline{b}$ mittelwertfrei sind, ist der Cosinus-Ähnlichkeitswert gleich dem Pearson-Korrelationswert. In der Dokument- und Textanalyse wird vornehmlich das Cosinus-Ähnlichkeitsmaß verwendet. 


__Russel Rao Ähnlichkeitsmaß__
Die Russel Rao-Ähnlichkeit zwischen zwei binären Vektoren $\underline{a}$ und $\underline{b}$ mißt das Verhältnis zwischen der Anzahl $\alpha$ der Stellen in denen beide Vektoren den Wert 1 haben und der Länge $n$ der Vektoren. Z.B. ist für die Vektoren $\underline{a}=(1,0,1,0,0,1)$ und $\underline{b}=(0,1,1,1,0,1)$ die Russel-Rao-Ähnlichkeit $s_{RR}(\underline{a},\underline{b})=2/6=0.333$.

__Jaccard Ähnlichkeitsmaß__
Die Jaccard-Ähnlichkeit zwischen zwei binären Vektoren $\underline{a}$ und $\underline{b}$ mißt das Verhältnis zwischen der Anzahl $\alpha$ der Stellen in denen beide Vektoren den Wert $1$ haben und der Anzahl der Stellen in denen mindestens einer der beiden Vektoren ungleich $0$ ist. Z.B. ist für die Vektoren $\underline{a}=(1,0,1,0,0,1)$ und $\underline{b}=(0,1,1,1,0,1)$ die Jaccard-Ähnlichkeit $s_{J}(\underline{a},\underline{b})=2/5=0.4$. Die Jaccard Metrik wird in diesem Versuch für die Bestimmung der Ähnlichkeit von *last.fm*-Usern eingesetzt.


## Vor dem Versuch zu klärende Fragen
Eine Untermenge der im Folgenden aufgeführten Fragen wird zu Beginn des Versuchs im Rahmen eines Gruppenkolloqs abgefragt. Auf jede Frage sollte von mindestens einem Gruppenmitglied eine Antwort geliefert werden und jedes Gruppenmitglied muss mindestens eine der gestellten Fragen beantworten können.

**Aufgaben:**

* Beschreiben Sie das Prinzip des userbasierten Collaborativen Filtering (UCF).

**Antwort**:

Das traditionelle **userbasierte Collaborative Filtering (UCF)**, benutzt die Ähnlichkeit zwischen Benutzern: Um für User $U_i$ eine Empfehlung zu erzeugen wird zunächst der diesem User ähnlichste Kunde (oder eine Menge vom ähnlichsten Kunden) ermittelt. Dann werden $U_i$ die Produkte (Items) empfohlen, welche der ähnlichste Kunde gekauft hat, $U_i$ selbst jedoch noch nicht. 

* Welche Nachteile hat das UCF?

**Antwort**:

Dieser Ansatz skaliert schlecht im Fall sehr großer *User/Item*-Matrizen. Ausserdem ist er für User, welche erst wenige Produkte gekauft haben unzuverlässig.

* Worin besteht der Unterschied zwischen UCF und itembasierten Collaborativen Filtering (ICF)?

**Antwort**:

Im Gegensatz zum UCF, werden beim ICF die Ähnlichkeit bezüglich der Items ermittelt und nicht die der User.

Gegeben seien die Vektoren 

$a=[1,2,3,4,5,6]$
    
$b=[3,3,5,6,7,8]$

Schreiben Sie eine Python Funktion, die den Mittelwert derartiger Vektoren berechnet. Schreiben Sie eine weitere Funktion, die die Varianz berechnet

In [1]:
import numpy as np
from pprint import pprint
import pandas as pd

from IPython.display import Latex
from IPython.display import Image
import pylast

In [2]:
a = np.array([1, 2, 3, 4, 5, 6])
b = np.array([3, 3, 5, 6, 7, 8])

In [3]:
def calc_mean(a, b):
    return np.mean(a), np.mean(b)


def calc_var(a, b):
    return np.var(a), np.var(b)

In [4]:
print(calc_mean(a, b))
print(calc_var(a, b))

(np.float64(3.5), np.float64(5.333333333333333))
(np.float64(2.9166666666666665), np.float64(3.5555555555555554))


* Wie groß ist die

    - Euklidische Ähnlichkeit
    - Pearson Ähnlichkeit
    - Cosinus Ähnlichkeit
    
    zwischen den Vektoren $\underline{a}$ und $\underline{b}$? 

In [5]:
def calc_euclidean_similarity(a, b):
    distance = np.sqrt(np.sum(np.square(a - b)))
    similarity = 1 / (1 + distance)
    return similarity

In [6]:
def calc_pearson_similarity(a, b):
    a1 = (a - np.mean(a)) / np.std(a)
    b1 = (b - np.mean(b)) / np.std(b)

    corr = 1/len(a) * np.sum(a1 * b1)
    return corr

In [7]:
def calc_cosinus_similarity(a, b):
    r1 = (a @ b)
    r2 = np.sqrt(np.sum(np.square(a))) * np.sqrt(np.sum(np.square(b)))
    return r1 / r2

In [8]:
calc_pearson_similarity(a, b)

np.float64(0.9833434220628547)

In [9]:
calc_euclidean_similarity(a, b)

np.float64(0.179128784747792)

In [10]:
calc_cosinus_similarity(a, b)

np.float64(0.991060084745164)

* In welchen Fällen sind Cosinus- und Pearsonähnlichkeit der euklidischen Ähnlichkeit vorzuziehen?

**Antwort**:

Für später

# Versuchsdurchführung
## Teil 1: Fiktive Filmbewertung
### Daten
Folgende Tabelle enthält die Filmbewertungen von 7 Personen.
from IPython.display import Latex
In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:


<img src="https://maucher.home.hdm-stuttgart.de/Pics/recommenderFilmRecommendations.PNG" style="width:1000px" align="center">



Die Tabelle ist als Python dictionary _critics_ implementiert. Die Keys des Python-Dictionary definieren die Namen von Personen (Zeilen in der Matrix), die Filme bewertet haben. Die Values sind selbst wieder Dictionarys, welche als Keys die Filmnamen (Spalten in der Matrix) und als Values die jeweilige Filmbewertung (Matrixelment) enthalten.

In [11]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}
}

### Ähnlichkeiten berechnen

Für die Bestimmung der Ähnlichkeit zwischen Personen und Produkten werden in diesem Versuch ein auf der euklidischen Distanz basierendes Ähnlichkeitsmaß und die Pearson-Korrelation verwendet. Beide Ähnlichkeitsmaße sind in den unten definierten Funktionen implementiert. Alle drei hier implementierten Funktionen zur Berechnung der Ähnlichkeit erhalten als Übergabeparameter das oben definierte Dictionary, das die Filmbewertungen enthält und die Namen der zwei Personen, die verglichen werden sollen. 

Zu beachten ist, dass in beiden Funktionen für die Berechnung der Ähnlichkeit zwischen zwei Personen nur die Produkte berücksichtigt werden, welche von beiden Personen schon bewertet wurden. Es handelt sich hier also um modifizierte Ähnlichkeitsfunktionen. 

__Aufgabe:__
Fragen Sie von diesem Dictionary _Toby's_ Bewertung des Films _Snakes on a Plane_ ab und geben Sie diesen Wert aus: 

In [12]:
critics['Toby']['Snakes on a Plane']

4.5

In [13]:
def sim_euclid(prefs,person1,person2,normed=True):
    ''' Returns a euclidean-distance-based similarity score for 
    person1 and person2. In the distance calculation the sum is computed 
    only over those items, which are nonzero for both instances, i.e. only
    films which are ranked by both persons are regarded.
    If the parameter normed is True, then the euclidean distance is divided by
    the number of non-zero elements integrated in the distance calculation. Thus
    the effect of larger distances in the case of an increasing number of commonly ranked
    items is avoided.
    '''
    # Get the list of shared_items
    si={}
    for item in prefs[person1]: 
        if item in prefs[person2]: 
            si[item] = 1
            
    # len(si) counts the number of common ratings
    # if they have no ratings in common, return 0
    if len(si) == 0: 
        return 0

    # Add up the squares of all the differences
    sum_of_squares=np.sqrt(sum([pow(prefs[person1][item]-prefs[person2][item],2) 
                     for item in prefs[person1] if item in prefs[person2]]))
    if normed:
        sum_of_squares= 1.0/len(si)*sum_of_squares
    return 1/(1+sum_of_squares)


def sim_pearson(prefs,p1,p2):
    '''
    Returns the Pearson correlation coefficient for p1 and p2
    '''

    # Get the list of commonly rated items
    si={}
    for item in prefs[p1]: 
        if item in prefs[p2]: 
            si[item]=1

    # if they are no ratings in common, return 0
    if len(si)==0: return 0

    # Sum calculations
    n=len(si)

    # Calculate means of person 1 and 2
    mp1=np.mean([prefs[p1][it] for it in si])
    mp2=np.mean([prefs[p2][it] for it in si])

    # Calculate standard deviation of person 1 and 2
    sp1=np.std([prefs[p1][it] for it in si])
    sp2=np.std([prefs[p2][it] for it in si])

    # If all elements in one sample are identical, the standard deviation is 0. 
    # In this case there is no linear correlation between the samples
    if sp1==0 or sp2==0:
        return 0
    r=1/(n*sp1*sp2)*sum([(prefs[p1][it]-mp1)*(prefs[p2][it]-mp2) for it in si])
    return r


def sim_RusselRao(prefs,person1,person2,normed=True):
    ''' Returns RusselRao similaritiy between 2 users. The RusselRao similarity just counts the number
    of common non-zero components of the two vectors and divides this number by N, where N is the length
    of the vectors. If normed=False, the division by N is omitted.
    '''
    # Get the list of shared_items
    si={}
    commons=0
    for item in prefs[person1]: 
        if prefs[person1][item]==1 and prefs[person2][item]==1:   
            commons+=1
    #print commons
    if not normed:
        return commons
    else:
        return commons*1.0/len(prefs[person1]) 

**Aufgabe:**
1. Geben Sie die euklidische Ähnlichkeit und die Pearson Ähnlichkeit zwischen den Personen _Toby_ und _Lisa Rose_ aus.
2. Diskutieren Sie die unterschiedlichen Ähnlichkeitswert.

In [14]:
print(sim_euclid(critics, 'Toby', 'Lisa Rose'))
print(sim_pearson(critics, 'Toby', 'Lisa Rose'))

0.615911621788925
0.9912407071619302


**Antwort**:
pass

__Aufgabe:__
0. Schreiben Sie eine Funktion `topMatches(prefs,person,similarity)`, welche für eine beliebige in _critics_ enthaltene Person die Ähnlichkeitswerte zu allen anderen Personen berechnet und in einer geordneten Liste zurück gibt. Der Funktion soll als Übergabeparameter auch die anzuwendende Ähnlichkeitsfunktion (`sim_euclid` oder `sim_pearson`) übergeben werden können. Berechnen Sie mit dieser Funktion für jede Person die *top matches*, zunächst unter Verwendung der euklidischen- dann unter Verwendung der Pearson-Ähnlichkeit.
1. Geben Sie mit der implementierten Funktion die *top matches* der Person Toby aus.
2. Vergleichen Sie die beiden Ähnlichkeitsmaße. Welches Ähnlichkeitsmaß erscheint Ihnen für diesen Anwendungsfall sinnvoller und warum?

In [15]:
def topMatches(prefs, person, similarity):

    temp_list = []
    
    if similarity == 'euclidic':
        for pers in prefs:
            # temp_list.append((f'{person} and {pers}', sim_euclid(prefs, person, pers)))
            temp_list.append(sim_euclid(prefs, person, pers))


    elif similarity == 'pearson':
        for pers in prefs:
            # temp_list.append((f'{person} and {pers}', sim_pearson(prefs, person, pers)))
            temp_list.append(sim_pearson(prefs, person, pers))

    
    return sorted(temp_list, reverse=True)
    # return temp_list

In [16]:
print('Euclidic Similarities:')
topMatches(critics, 'Toby', 'euclidic')

Euclidic Similarities:


[np.float64(1.0),
 np.float64(0.6666666666666666),
 np.float64(0.6246387977050463),
 np.float64(0.615911621788925),
 np.float64(0.5584815598877471),
 np.float64(0.5227744249483389),
 np.float64(0.5108747069239427)]

In [17]:
print('Pearson Similarities:')
topMatches(critics, 'Toby', 'pearson')

Pearson Similarities:


[np.float64(0.9999999999999999),
 np.float64(0.9912407071619302),
 np.float64(0.9244734516419051),
 np.float64(0.8934051474415642),
 np.float64(0.6628489803598703),
 np.float64(0.3812464258315117),
 np.float64(-1.0)]

**Antwort**:

pass

### Berechnung von Empfehlungen mit User basiertem Collaborative Filtering
Für die Produkte, die von einer Person noch nicht gekauft wurden, sollen Empfehlungen berechnet werden. Die Empfehlungen können ebenfalls Werte zwischen 1 (wird nicht empfohlen) und 5 (wird stark empfohlen) annehmen. Für die Berechnung der Empfehlung werden die Bewertungen des jeweiligen Produkts durch die anderen Personen herangezogen. Dabei werden die Bewertungen der ähnlichen Personen (d.h. hoher Pearson-Korrelationswert) stärker mit einbezogen als die Bewertungen durch Personen mit einem niedrigen Korrelationswert.

__Beispiel:__
Toby hat die Filme *The Night Listener*, _Lady in the Water_ und _Just My Luck_ noch nicht gekauft. Für diese Filme soll für Toby eine Empfehlung berechnet werden.
In der unten aufgeführten Tabelle enthält die zweite Spalte die _Pearson-Ähnlichkeitswerte_ zwischen Toby und den anderen Personen. Die Spalten 3, 5 und 7 enthalten die Bewertungen der Filme *The Night Listener*, _Lady in the Water_ und _Just My Luck_ durch die anderen Personen. Die Spalten 4, 6 und 8 enthalten die jeweilige Filmbewertung gewichtet (mulipliziert) mit den Ähnlichkeitswerten der jeweiligen Person. Es fällt auf, dass in der Tabelle _Michael_ nicht enthalten ist. Das liegt daran, dass _Michael_ und _Toby_ einen negativen Ähnlichkeitswert aufweisen, d.h. deren Interessen sind gegenläufig. Personen mit negativem Ähnlichkeitswert sollten für Empfehlungen nicht berücksichtigt werden.
Die Zeile _Sum_ enthält die Summe aller gewichteten Bewertungen. Aus diesem Wert allein kann die Empfehlung noch nicht abgeleitet werden, da Filme die nur von wenigen Personen bewertet wurden, eine relativ kleine Summe ergeben. Deshalb sollte _Sum_ noch durch die Anzahl der Bewertungen für diesen Film geteilt werden. Oder besser: Nicht durch die Summe der Bewertungen, sondern durch die Summe der relevanten Ähnlichkeitswerte (*KSum*). Der resultierende Empfehlungswert ist in der letzten Zeile eingetragen.



<img src="https://maucher.home.hdm-stuttgart.de/Pics/recommenderFilmCalculation.PNG" style="width:1000px" align="center">




__Aufgabe:__
Schreiben Sie eine Funktion *getRecommendations(prefs,person,similarity)*, mit der die Empfehlungswerte berechnet werden können und bestimmen Sie die Empfehlungswerte für Toby. Der Funktion wird  

* das Dictionary _critics_ mit den Filmbewertungen, 
* der Name der Person, für welche Empfehlungen berechnet werden sollen
* die Methode für die Berechnung der Ähnlichkeit *sim_euclid* oder *sim_pearson*

übergeben. Die Methode soll eine geordnete Liste zurück geben. Jedes Listenelement enthält an erster Stelle den berechneten Empfehlungswert und an zweiter Stelle den Namen des Films. Die Liste soll nach Empfehlungswerten absteigend geordnet sein.

Testen Sie diese Funktion indem Sie die Empfehlungen für _Toby_ berechnen und mit den Werten in der oben aufgeführten Tabelle vergleichen.

In [18]:
def getRecommendations(prefs, person, similarity):
    ''' Return: [(EW, NF), ..] '''
    films_person_not_seen = [j for j in list(set([i for i in prefs for i in prefs[i]])) if j not in prefs[person]]
    # print(films_person_not_seen)

    corr_with_other = topMatches(prefs, person, similarity)[:-1]
    corr_with_other = [i for i in corr_with_other if i[1] >= 0]
    # print(corr_with_other)  

    mean_other_pers = []

    for film in films_person_not_seen:
        mean_other_pers.append(np.array([prefs[pers][film] if film in prefs[pers].keys() and sim_pearson(prefs, person, pers) >= 0 else np.nan for pers in prefs]))
    # print(mean_other_pers)

    x = [[], [], [], [], [], [],[]]

    for i in mean_other_pers:
        for index, j in enumerate(i):
            x[index].append(j)
    # print(x)

    corr_film = [i for i in x if np.isnan(i).all() == False]
    corr_film1 = [(np.array(corr_film[index]) * corr[1], corr[1]) for index, corr in enumerate(corr_with_other)]
    corr_film2 = [np.array(corr_film[index]) * corr[1] for index, corr in enumerate(corr_with_other)]
    # pprint(corr_film2)

    corr_film_sums = [np.nansum(group) for group in zip(*corr_film2)]
    # print(corr_film_sums)

    k_sum = []
    for i in range(3):
        c = [j[1] if not np.isnan(j[0][i]) else '' for j in corr_film1]
        c = [i for i in c if i != '']
        k_sum.append(np.sum(c))
    
    # print(k_sum)

    sum_k_sum = np.array(corr_film_sums) / np.array(k_sum)

    result = sorted([(value, title) for value, title in zip(sum_k_sum, films_person_not_seen)], key=lambda x: x[0], reverse=True)
    return result

In [19]:
def getRecommendations(prefs, person, similarity, task):

    # Choose the task, because of the key-selection-methode of the tasks
    if task == 'movie':
        all_keys = list(set([item for sublist in [prefs[i].keys() for i in prefs] for item in sublist]))
        keys_for_reccomendation = [i for i in all_keys if i not in prefs[person].keys()]
    elif task == 'fm':
        keys_for_reccomendation = [artist for artist in prefs[person] if prefs[person][artist] == 0]

    # Choose the similarity metric
    if similarity == 'euclidic':
        similarity_values = [[pers, sim_euclid(prefs, person, pers)] for pers in prefs]
        similarity_values = [value for value in similarity_values if value[0] != person]

        for pers in similarity_values:
            if sim_euclid(prefs, person, pers[0]) < 0:
                del prefs[pers[0]]

    elif similarity == 'pearson':
        similarity_values = [[pers, sim_pearson(prefs, person, pers)] for pers in prefs]
        similarity_values = [value for value in similarity_values if value[0] != person]

        for pers in similarity_values:
            if sim_pearson(prefs, person, pers[0]) < 0:
                del prefs[pers[0]]
    
    # Collect the ratings of the ther persons/users
    ratings_by_other = [np.array([prefs[pers][title] if title in prefs[pers].keys() else np.nan for pers in prefs]) for title in keys_for_reccomendation]
 
    # Stack the array for better calculation
    # Remove arrays consisiting of nans
    # Calculate the ratings with the similarities
    stacked_raitngs_vector = np.column_stack(ratings_by_other)
    filtered_a = [arr for arr in stacked_raitngs_vector if not np.all(np.isnan(arr))]
    k_multiply_ratings = [value * sim[1] for value, sim in zip(filtered_a, similarity_values)]
    
    # Unstack the srrays fot better calculation
    # Calculate the sum of the calculations above
    unstacked_a = list(zip(*filtered_a))
    k_multiply_ratings_sum = [np.nansum(i) for i in list(zip(*k_multiply_ratings))]

    # Calculate the sum of the used similarities
    k_sum = []
    for rating in unstacked_a:
        nan_indices = np.where(np.isnan(rating))[0]
        k_sum.append(np.sum([i[1] for index, i in enumerate(similarity_values) if index not in nan_indices]))

    # Calculate the desission value
    sum_k_sum = np.array(k_multiply_ratings_sum) / np.array(k_sum)

    # Return a sorted list with the decission value und and the title
    result = sorted([(value, title) for value, title in zip(sum_k_sum, keys_for_reccomendation)], key=lambda x: x[0], reverse=True)
    return result

In [20]:
getRecommendations(critics, 'Toby', 'pearson', 'movie')

[(np.float64(2.3553000598065337), 'Lady in the Water'),
 (np.float64(1.7773275786524223), 'The Night Listener'),
 (np.float64(1.2092871669862821), 'Just My Luck')]

<!-- ### Berechnung von Empfehlungen mit Item basiertem Collaborative Filtering
In den vorigen Aufgaben wurden Ähnlichkeiten zwischen Personen bestimmt und für Produktempfehlungen benutzt (User basiertes Collaborative Filtering). Jetzt soll die Ähnlichkeit zwischen Produkten berechnet werden und auf der Basis dieser Produktähnlichkeit Empfehlungen berechnet werden (Item basiertes Collaborative Filtering).

Dabei sollen die bereits implementierten Ähnlichkeitsfunktion *sim_euclid* und *sim_pearson* sowie die Ähnlichkeeits-Sortierfunktion *topMatches* unverändert eingesetzt werden.

__Aufgabe:__

1. Implementieren Sie eine Funktion, welche das Bewertungsdictionary *critics* derart transformiert, dass die Funktionen `sim_euclid`, `sim_pearson` und `topMatches` für das Item-basierte CF unverändert eingesetzt werden können. Die transformierte Matrix soll unter dem Namen *transCritics* abgespeichert werden.
2. Schreiben Sie eine Funktion `calculateSimilarItems`, die aus der transformierten Matrix *transCritics* ein Dictionary berechnet, welches die Ähnlichkeit zwischen allen Filmen beschreibt. Die Keys des Dictionary sind die Filmnamen. Die Values sind geordnete Listen, welche die Funktion `topMatches` zurückgibt, wenn sie für die Filme (nicht für die User) aufgerufen wird. Dieses Dictionary wird an das aufrufende Programm zurück geben. 
3. Schreiben Sie eine Funktion `getRecommendedItems`, welche basierend auf dem im unten aufgeführten Beispiel dargestellten Verfahren unter Vorgabe der Bewertungsmatrix und der zu verwendenden Ähnlichkeitsfunktion Produktempfehlungen berechnet.
4. Testen Sie die Funktion indem Sie die Empfehlungen für Toby berechnen und mit den Werten in der unten aufgeführten Tabelle vergleichen

__Erläuterndes Beispiel:__

_Toby_ hat die Filme *The Night Listener*, *Lady in the Water* und *Just My Luck* noch nicht gekauft. Für diese Filme soll für *Toby* eine Empfehlung berechnet werden. Gekauft und bewertet hat *Toby* die Filme *Snakes on a plane*, *Superman Returns* und *You and me and Dupree*. Diese bereits vorhandenen Filme bilden die erste Spalte der unten dargestellten Matrix. In der zweiten Spalte befinden sich _Toby's_ Bewertungen dieser Filme. Die Spalten 3,5 und 7 enthalten die Ähnlichkeitswerte (mit *calculateSimilarItems* unter Verwendung des normierten euklidischen Ähnlichkeitsmaßes berechnet) zwischen den drei von *Toby* noch nicht gekauften Filmen und den drei von _Toby_ bewerteten Filmen. Diese Ähnlichkeitswerte werden jeweils mit _Toby's_ Bewertungen multipliziert. Das Resultat dieser Multiplikation befindet sich in den Spalten 4,6 und 8. Der finale Empfehlungswert für die von _Toby_ noch nicht gekauften Filme wird berechnet in dem in den Spalten 4,6 und 8 zunächst die Summe über die Werte dieser Spalte in den drei oberen Zeilen berechnet wird und durch die Summe über die Werte der Spalten 3,5 und 7 geteilt wird. Im Fall, dass die *Pearson-Korrelation* zwischen den Filmen als Ähnlichkeitswert herangezogen wird, können negative Ähnlichkeitswerte auftreten. Dann soll in die Berechnung eines Empfehlungswert für Film A nur dann die Bewertung von Film B einfließen, wenn der Korrelationswert zwischen beiden $>0$ ist.  


<img src="https://maucher.home.hdm-stuttgart.de/Pics/recommenderFilmItemBased.PNG" style="width:500px" align="center"> -->


## last.fm Musikempfehlungen

__Aufgabe:__

1. Installieren Sie das Package pylast. Stellen Sie durch Aufruf der Funktion *network=pylast.LastFMNetwork()* eine Verbindung zu *last.fm* her. Beim Aufruf der Funktion wird die Anmeldung und Authentifizierung durchgeführt. Die Funktion gibt ein Objekt der Klasse *Network* zurück. Über dieses Objekt können Methoden, wie

    * *get_artist("kuenstlerName")* (liefert Objekt der Klasse *Artist*)
    * *get_album("albumName")* (liefert Objekt der Klasse *Album*)
    * *get_track("songName")* (liefert Objekt der Klasse *Track*)
    * *get_user("userName"):* (liefert Objekt der Klasse *Tag*)
    * usw.
    
      aufgerufen werden. Die Menge aller verfügbaren Klassen und deren Attribute und Methoden können dem Modul _pylast.py_ entnommen werden. In der untenstehenden Codel Zelle können Sie beispielsweise über das *Network*-Objekt die Methode `get_artist("BandIhrerWahl")` aufrufen um die von lastFM berechneten ähnlichsten Bands anzuzeigen.

2. Führen Sie die untenstehende Code Zelle aus um eine vordefinierte Gruppe von Benutzern, passend zu Band 'Slipknot' zu bekommen. Falls Sie ein lastfm Konto haben können Sie manuell auch andere Bands und Benutzer eintragen.
3. Implementieren Sie eine Funktion `createLastfmUserDict()`. Dieser Funktion soll, die oben angelegte Liste von *User*-Objekten _group_ übergeben werden. Für jeden User in *group* sollen die 20 beliebtesten Bands mit der Methode `topartists=*User*.get_top_artists()[0:20]` bestimmt werden. Die Methode gibt eine Liste von *Artist*-Objekt/Gewichtung-Paaren zurück. Die Gewichtungen von Objekten werden in diesem Versuch nicht benötigt. Auf das *i.te* *Artist*-Objekt selbst kann mit `topartists[i].item` zugegriffen werden. Die Menge aller Bands, die auf diese Weise gesammelt werden, wird im folgenden mit _AllBands_ bezeichnet. D.h. in *AllBands* befinden sich alle Bands, die für mindestens einen User in *group* zu den Top-20 gehören. Nun soll ein verschachteltes Dictionary mit Namen *userDict* wie folgt angelegt werden:

    * Die Keys sind die Namen der _User_-Objekte in _group_. Auf den Namen eines Objekts kann mit `get_name()` zugegriffen werden.
    * Die Values sind selbst wieder Dictionaries, deren Keys die Namen der Bands in *AllBands* sind. Achten Sie auch hier darauf, dass Sie nicht das *Artist*-Objekt selbst, sondern dessen Namen als Key verwenden. 
    * Für den User *a* und die Band *b* ist der Value `userDict[a][b]= 1`, falls *b* zu den Top-20 des Users *a* gehört. Andernfalls ist `userDict[a][b]= 0`. 
    
    Das derart angelegte Dictionary soll von der Funktion zurückgegeben werden. 
4. Wählen Sie jetzt einen beliebigen User aus *group*. Bestimmen Sie zu diesem User die ähnlichsten User in *group* durch Anwendung der im ersten Teilversuch implementierten Funktion `topMatches()`. Der Funktion wird das angelegte *userDict* und der Name des gewählten Users übergeben. Als Ähnlichkeitsmaß soll die euklidische Metrik angewandt werden.
5. Bestimmen Sie dann für den gewählten User Band-Empfehlungen durch Anwendung der im ersten Teilversuch implementierten Funktion `getRecommendations()`. Der Funktion wird das angelegte *userDict* und der Name des gewählten Users übergeben. Als Ähnlichkeitsmaß soll die euklidische Metrik, danach die Russel_Rao Metrik, angewandt werden.     
6. Diskutieren Sie das Ergebnis

In [21]:
nw=pylast.LastFMNetwork(api_key="993a5bd9d79a98a53677570368d55acd", api_secret="9b8de0b57903ac007cdd8ec9003b341e", username="pythonlab")
band='Slipknot'
art1 = nw.get_artist(band)
print("Most similar (as calculated by lastFM) for artist: ", band)

for it in art1.get_similar(5):
    print("%3.3f \t %s"%(it.match, it.item))

print("\nApply predefined group of users")
usernames=['BrunoJoS','DPREBOYE','MPistol40','NemoNightfall','SkyRif','Wags1382','Znapsen','cortapsyco','emill_67','sattuviitana']
group=[]
for u in usernames:
    u1 = nw.get_user(u)
    group.append(u1)
print(group)

Most similar (as calculated by lastFM) for artist:  Slipknot
1.000 	 Korn
0.727 	 Stone Sour
0.701 	 System of a Down
0.564 	 Mudvayne
0.533 	 Vended

Apply predefined group of users
[pylast.User('BrunoJoS', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('DPREBOYE', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('MPistol40', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('NemoNightfall', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('SkyRif', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('Wags1382', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b34

In [22]:
def createLastfmUserDict(group):
    AllBands = []

    for user in group:
        topartists = user.get_top_artists()[0:20]
        for artist in topartists:
            AllBands.append(artist.item.name)

    userDict = {user.get_name():{band:0 for band in AllBands} for user in group}

    for user, user_obj in zip(userDict, group):
        topartists = user_obj.get_top_artists()[:20]
        for artist in topartists:
            if artist.item.name in userDict[user].keys():
                userDict[user][artist.item.name] = 1

    return userDict  

In [23]:
user_dict = createLastfmUserDict(group)

In [24]:
user_dict

{'BrunoJoS': {'Muse': 1,
  'Slipknot': 1,
  'Queens of the Stone Age': 1,
  'Arctic Monkeys': 1,
  'Linkin Park': 1,
  'Radiohead': 1,
  'Rammstein': 1,
  'Papa Roach': 1,
  'Foo Fighters': 1,
  'Sum 41': 1,
  'Green Day': 1,
  'The Killers': 1,
  'System of a Down': 1,
  'Royal Blood': 1,
  '30 Seconds to Mars': 1,
  'Madonna': 1,
  'Pitty': 1,
  'Three Days Grace': 1,
  'Lady Gaga': 1,
  'M.I.A.': 1,
  'Warpaint': 0,
  'DIR EN GREY': 0,
  'The White Stripes': 0,
  'James Blake': 0,
  'Daughter': 0,
  'Dr. Dre': 0,
  'Beach House': 0,
  'Atoms for Peace': 0,
  'Damien Rice': 0,
  'Die Antwoord': 0,
  'Mogwai': 0,
  'Rage Against the Machine': 0,
  'Hozier': 0,
  'The Band': 0,
  'Nirvana': 0,
  'Wu-Tang Clan': 0,
  'Cat Power': 0,
  'N.W.A': 0,
  'In Flames': 0,
  'Deftones': 0,
  'Korn': 0,
  'Red Hot Chili Peppers': 0,
  'Dark Tranquillity': 0,
  'Soilwork': 0,
  'Limp Bizkit': 0,
  '36 Crazyfists': 0,
  'Soulfly': 0,
  'Stone Sour': 0,
  'Scar Symmetry': 0,
  'Nine Inch Nails': 0,


In [25]:
topMatches(prefs=user_dict, person='DPREBOYE', similarity='euclidic')

[np.float64(1.0),
 np.float64(0.9615384615384615),
 np.float64(0.9615384615384615),
 np.float64(0.9615384615384615),
 np.float64(0.9615384615384615),
 np.float64(0.9605261285528738),
 np.float64(0.9605261285528738),
 np.float64(0.9605261285528738),
 np.float64(0.9605261285528738),
 np.float64(0.9605261285528738)]

In [26]:
getRecommendations(prefs=user_dict, person='DPREBOYE', similarity='euclidic', task='fm')

[(np.float64(0.5556465937934617), 'System of a Down'),
 (np.float64(0.44423635704351605), 'Korn'),
 (np.float64(0.33341136610868155), 'Linkin Park'),
 (np.float64(0.33341136610868155), 'Papa Roach'),
 (np.float64(0.3332943169456593), 'Stone Sour'),
 (np.float64(0.3332943169456593), 'Five Finger Death Punch'),
 (np.float64(0.33317726778263707), 'Red Hot Chili Peppers'),
 (np.float64(0.22235227684780254), '30 Seconds to Mars'),
 (np.float64(0.22235227684780254), 'Three Days Grace'),
 (np.float64(0.22223522768478027), 'Muse'),
 (np.float64(0.22223522768478027), 'Rammstein'),
 (np.float64(0.22223522768478027), 'Foo Fighters'),
 (np.float64(0.22223522768478027), 'Pendulum'),
 (np.float64(0.22223522768478027), 'Coheed and Cambria'),
 (np.float64(0.22211817852175803), 'Limp Bizkit'),
 (np.float64(0.22211817852175803), 'Lamb of God'),
 (np.float64(0.22211817852175803), 'Metallica'),
 (np.float64(0.22211817852175803), 'Pink Floyd'),
 (np.float64(0.11117613842390127), 'Queens of the Stone Age'),